In [77]:
import importlib
from typing import List, Tuple
import pandas as pd
import numpy as np

from constants import *
import utils
importlib.reload(utils)

<module 'utils' from '/mnt/cb8c2a81-32e7-40bc-af74-0c1386cda683/programowanie/pipelon/Classifier-Builder/lol_classifier/utils.py'>

In [78]:
from glob import glob
import os

datasets = glob(os.path.join(DATASET_PATH, f"*/{DATASET_POSTFIX}"))
print(f"Found {len(datasets)} datasets")

Found 10 datasets


In [79]:
utils.clear_directories([TRAINING_DATA_PATH, TESTING_DATA_PATH])
for path in datasets:
  filename = path.split('/')[-2]

  df = utils.read_eeg_data(path)
  df.drop(df[df.iloc[:, -1] == 'd'].index, inplace=True)
  markers = df.iloc[:, -1]
  print(f"From {filename} with shape:{df.shape} and markers:{len(markers[markers != '0'])} kills:{len(markers[markers == 'kill'])}, deaths:{len(markers[markers == 'death'])}")

  prev_marker_and_countdown = ['0', 0]
  df.iloc[:, -1] = markers.apply(lambda x: utils.propagate_events(x, prev_marker_and_countdown))
  
  df.drop(df[markers == '0'].index, inplace=True)  
  df = df.sample(frac=1, random_state=42).reset_index(drop=True)
  utils.split_and_save(df, f"{filename}.csv")
  

From lol_2024-08-04T21-10-29 with shape:(41331, 41) and markers:7 kills:0, deaths:7
From lol_2024-08-08T14-29-23 with shape:(61232, 41) and markers:9 kills:0, deaths:9
From lol_2024-08-08T16-38-51 with shape:(49694, 41) and markers:20 kills:10, deaths:10
From lol_2024-08-08T19-40-51 with shape:(44081, 41) and markers:15 kills:8, deaths:7
From lol_2024-08-08T20-58-04 with shape:(30700, 41) and markers:11 kills:3, deaths:8
From lol_2024-08-08T21-44-59 with shape:(29347, 41) and markers:8 kills:2, deaths:6
From lol_2024-08-09T14-47-50 with shape:(54396, 41) and markers:16 kills:8, deaths:8
From lol_2024-08-12T11-51-55 with shape:(49782, 41) and markers:9 kills:5, deaths:4
From lol_2024-08-12T15-18-29 with shape:(53692, 41) and markers:9 kills:0, deaths:9
From lol_2024-08-12T17-29-12 with shape:(42603, 41) and markers:12 kills:8, deaths:4


In [80]:
train, test = utils.load_datasets()
train.shape, test.shape

Number of kills=2537; deaths=4239


((6776, 41), (1697, 41))

In [81]:
from sklearn.svm import SVC

X_train, y_train = train[:, :-1], train[:, -1]
X_test, y_test = test[:, :-1], test[:, -1]

svm = SVC(kernel='linear', C=1, random_state=42)
svm.fit(X_train, y_train)

SVC(C=1, kernel='linear', random_state=42)

In [82]:
print('accuracy:', svm.score(X_test, y_test))
svm.predict(X_test[0, :].reshape(1, -1))

accuracy: 0.7289334119033589


array([1.])